In [1]:
from yolox.tracker.byte_tracker import BYTETracker
from yolox.utils.visualize import plot_tracking
# from bytetrack import BYTETracker
from ultralytics.utils import LOGGER
from ultralytics import YOLO
import cv2

import numpy as np
import torch

# yolo 로그 메시지 비활성화
LOGGER.disabled = True

In [2]:
# YOLOv8 모델 로드 (사전 학습된 COCO 데이터셋 사용)
model = YOLO('yolov8n.pt')  # 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt' 중 선택 가능

In [3]:
# ByteTrack 설정
tracker_args = {
    'track_thresh': 0.3,
    'match_thresh': 0.3,
    'track_buffer': 30,
    'mot20': False
}

# 딕셔너리를 Namespace로 변환
from argparse import Namespace
tracker_args = Namespace(**tracker_args)

# # BYTETracker 클래스 내부에서 딕셔너리 처리
# class BYTETracker:
#     def __init__(self, args, frame_rate=30):
#         # 딕셔너리로 처리
#         self.det_thresh = args['track_thresh'] + 0.1
#         self.buffer_size = int(frame_rate / 30.0 * args['track_buffer'])
#         self.max_time_lost = self.buffer_size
#         # 기타 초기화 코드

tracker = BYTETracker(tracker_args)

In [4]:
# 2. 로컬 영상 파일 열기
video_path = '../../../Data/video/1-1_006-C04.mp4'
url = 'rtsp://210.99.70.120:1935/live/cctv002.stream'
cap = cv2.VideoCapture(url)

In [5]:
# 3. 프레임별 처리
object_tracks = {}  # 객체 위치 저장용 딕셔너리
TARGET_CLASS = [2, 5, 7]
count_num = 0

frame_count = 0
skip_frames = 5  # 5프레임마다 한 번씩 처리

print("영상시작")

# 첫 번째 프레임에서 프레임 크기 초기화
ret, frame = cap.read()
if not ret:  # 첫 번째 프레임 읽기 실패 시 프로그램 종료
    print("비디오 파일을 읽을 수 없습니다.")
    cap.release()
    exit()

frame_height, frame_width = frame.shape[:2]
mid_x = frame_width // 2  # 화면 중간 x좌표 계산
mid_y = frame_height // 2  # 화면 중간 y좌표 계산

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % skip_frames != 0:  # 프레임 스킵
        continue
    if frame_count % 1000 == 0:  # 1000프레임마다 스트림 재연결
        print("스트림 초기화")
        cap.release()
        cap = cv2.VideoCapture(url)

    # YOLOv8 객체 탐지
    results = model(frame)
    detections = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            confidence = float(box.conf[0])  # tensor -> float 변환  # 신뢰도
            class_id = int(box.cls[0])  # 클래스id
            if confidence > 0.5:  # 신뢰도 필터링
                detections.append([x1, y1, x2, y2, confidence, class_id])  # [x1, y1, x2, y2, confidence, class_id]
    
    # ByteTrack 추적
    img_info = (frame_height, frame_width)  # 이미지 정보
    img_size = (frame_width, frame_height)  # 이미지 크기

    if len(detections) == 0:
        detection_array = np.empty((0, 5), dtype=float)  # 빈 배열 생성
    else:
        detection_array = np.array(detections, dtype=float)  # 넘파이 버전 이슈로 dtype수정
    print(f"detections: {detections}")
    detection_tensor = torch.from_numpy(detection_array).type(torch.float32)  # numpy 배열을 PyTorch Tensor로 변환
    online_targets = tracker.update(detection_tensor, img_info, img_size)
    
    # 추적 결과 시각화
    for target in online_targets:
        x1, y1, w, h = target.tlwh
        track_id = target.track_id
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        


#         if confidence > 0.5 and class_id in TARGET_CLASS:
#             center_x = int((x1 + x2) / 2)
#             center_y = int((y1 + y2) / 2)

#             # object_id =  hash(f"{class_id}_{center_x}_{center_y}")  # 고유 ID 생성
#             object_id = f"{class_id}_{x1}_{y1}_{x2}_{y2}"
#             current_frame_tracks[object_id] = (center_x, center_y)

#             #이전 위치와 비교하여 이동 방향 계산
#             if object_id in object_tracks:
#                 prev_center_x, prev_center_y = object_tracks[object_id]
#                 print("object_id", object_id)
#                 print("object_tracks[object_id]", object_tracks[object_id])
#                 print("prev_center_x", prev_center_x)
#                 print("prev_center_y", prev_center_y)

#                  # 디버깅: 객체 이동 방향 확인
#                 print(f"객체 {object_id}: prev_center_y={prev_center_y}, center_y={center_y}, mid_y={mid_y}")

#                 # 위쪽-> 아래쪽 이동
#                 if prev_center_y < mid_y - threshold and center_y >= mid_y + threshold:
#                     count_num += 1
#                     print(f"객체 {object_id}가 위쪽에서 아래쪽으로 이동. 현재 인원수: {count_num}")

#                 # 아래쪽 -> 위쪽 이동
#                 elif prev_center_y > mid_y + threshold and center_y <= mid_y - threshold:
#                     count_num -= 1
#                     print(f"객체 {object_id}가 아래쪽에서 위쪽으로 이동. 현재 인원수: {count_num}")

#             # 현재 위치 업데이트
#             # object_tracks[object_id] = (center_x, center_y)
#             object_tracks.update(current_frame_tracks)

#             # 경계 상자 및 중심점 표시
#             cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
#             cv2.circle(frame, (center_x, center_y), 5, (255,0, 0), -1)
    
#     # # 객체 이동 방향 업데이트
#     object_tracks = current_frame_tracks            
    
    
#     # 영상 중간 부분에 세로선 그리기
#     # cv2.line(frame, (mid_x, 0), (mid_x, frame_width), (255, 0, 0), 2)  # (255, 0, 0)은 파란색, 두께는 2
    
    # 영상 중간 부분에 가로선 그리기
    cv2.line(frame, (0, mid_y), (frame_width, mid_y), (255, 0, 0), 1)  # (255, 0, 0)은 파란색, 두께는 2

    # 프레임 보여주기
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


영상시작
detections: []
detections: []


c:\miniconda3\envs\p2\lib\site-packages\yolox\tracker\byte_tracker.py:182: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  dets_second = bboxes[inds_second]
c:\miniconda3\envs\p2\lib\site-packages\yolox\tracker\byte_tracker.py:185: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:28.)
  scores_second = scores[inds_second]


detections: [[347, 240, 413, 304, 0.532392144203186, 2]]
detections: [[349, 231, 413, 293, 0.6255250573158264, 2]]
detections: []
detections: [[354, 215, 411, 273, 0.5963394641876221, 2]]
detections: [[356, 191, 414, 263, 0.5620891451835632, 2]]
detections: []
detections: [[361, 195, 414, 244, 0.5662673711776733, 2]]
detections: [[364, 187, 415, 238, 0.5865112543106079, 2]]
detections: [[367, 166, 417, 229, 0.6324102282524109, 2]]
detections: [[369, 164, 419, 221, 0.682759702205658, 2], [579, 275, 614, 336, 0.5111627578735352, 12]]
detections: [[371, 159, 420, 209, 0.5743495225906372, 2]]
detections: [[375, 151, 421, 204, 0.6751582026481628, 2]]
detections: [[378, 154, 421, 199, 0.5298340320587158, 2]]
detections: []
detections: [[384, 143, 423, 185, 0.5053769946098328, 2]]
detections: [[580, 276, 614, 336, 0.549763560295105, 12]]
detections: []
detections: []
detections: []
detections: []
detections: []
detections: []
detections: []
detections: []
detections: []
detections: []
detecti